# Relation extraction using distant supervision: experiments

In [34]:
__author__ = "Bill MacCartney and Christopher Potts"
__version__ = "CS224u, Stanford, Spring 2021"

## Contents

1. [Overview](#Overview)
1. [Set-up](#Set-up)
1. [Building a classifier](#Building-a-classifier)
  1. [Featurizers](#Featurizers)
  1. [Experiments](#Experiments)
1. [Analysis](#Analysis)
  1. [Examining the trained models](#Examining-the-trained-models)
  1. [Discovering new relation instances](#Discovering-new-relation-instances)

## Overview

OK, it's time to get (halfway) serious. Let's apply real machine learning to train a classifier on the training data, and see how it performs on the test data. We'll begin with one of the simplest machine learning setups: a bag-of-words feature representation, and a linear model trained using logistic regression.

Just like we did in the unit on [supervised sentiment analysis](https://github.com/cgpotts/cs224u/blob/master/sst_02_hand_built_features.ipynb), we'll leverage the `sklearn` library, and we'll introduce functions for featurizing instances, training models, making predictions, and evaluating results.

## Set-up

See [the first notebook in this unit](rel_ext_01_task.ipynb#Set-up) for set-up instructions.

In [113]:
from collections import Counter
import os
import rel_ext
import utils

In [114]:
# Set all the random seeds for reproducibility. Only the
# system seed is relevant for this notebook.

utils.fix_random_seeds()

In [115]:
rel_ext_data_home = os.path.join('data', 'rel_ext_data')

With the following steps, we build up the dataset we'll use for experiments; it unites a corpus and a knowledge base in the way we described in [the previous notebook](rel_ext_01_task.ipynb).

In [116]:
corpus = rel_ext.Corpus(os.path.join(rel_ext_data_home, 'washington_post_test.tsv.gz'))

In [119]:
kb = rel_ext.KB(os.path.join(rel_ext_data_home, 'Atsuko_filtered_KB.tsv.gz'))

In [120]:
def find_match(corpus):
        related_pairs = set()
        matches_found = []
        for ex in corpus.examples:
            if kb.get_triples_for_entities(ex.entity_1, ex.entity_2):
                related_pairs.add((ex.entity_1, ex.entity_2))
            if kb.get_triples_for_entities(ex.entity_2, ex.entity_1):
                related_pairs.add((ex.entity_2, ex.entity_1))
                
        for pair in related_pairs:
            match = kb.get_triples_for_entities(pair[0], pair[1])
            matches_found.append(match)
        return matches_found

In [121]:
matches = find_match(corpus)

In [122]:
len(matches)

399

In [123]:
matches[10: 20]

[[KBTriple(rel='adjoins', sbj='Africa', obj='Europe')],
 [KBTriple(rel='contains', sbj='New_York', obj='Bard_College')],
 [KBTriple(rel='contains', sbj='Israel', obj='Jerusalem'),
  KBTriple(rel='capital', sbj='Israel', obj='Jerusalem')],
 [KBTriple(rel='adjoins', sbj='Georgia', obj='South_Carolina')],
 [KBTriple(rel='contains', sbj='Hubei', obj='Xiantao')],
 [KBTriple(rel='contains', sbj='Bangladesh', obj='Dhaka'),
  KBTriple(rel='capital', sbj='Bangladesh', obj='Dhaka')],
 [KBTriple(rel='adjoins', sbj='Australia', obj='Indonesia')],
 [KBTriple(rel='adjoins', sbj='Saudi_Arabia', obj='United_Arab_Emirates')],
 [KBTriple(rel='contains', sbj='Queensland', obj='Brisbane'),
  KBTriple(rel='capital', sbj='Queensland', obj='Brisbane')],
 [KBTriple(rel='adjoins', sbj='Somalia', obj='Kenya')]]

In [124]:
dataset = rel_ext.Dataset(corpus, kb)

The following code splits up our data in a way that supports experimentation:

In [125]:
splits = dataset.build_splits(
    split_names=['train', 'test'],
    split_fracs=[0.80, 0.20],
    seed=1)
splits

{'train': Corpus with 54,237 examples; KB with 22,313 triples,
 'test': Corpus with 14,543 examples; KB with 6,262 triples,
 'all': Corpus with 68,780 examples; KB with 28,575 triples}

## Building a classifier

### Featurizers

Featurizers are functions which define the feature representation for our model. The primary input to a featurizer will be the `KBTriple` for which we are generating features. But since our features will be derived from corpus examples containing the entities of the `KBTriple`, we must also pass in a reference to a `Corpus`. And in order to make it easy to combine different featurizers, we'll also pass in a feature counter to hold the results.

Here's an implementation for a very simple bag-of-words featurizer. It finds all the corpus examples containing the two entities in the `KBTriple`, breaks the phrase appearing between the two entity mentions into words, and counts the words. Note that it makes no distinction between "forward" and "reverse" examples.

In [126]:
def simple_bag_of_words_featurizer(kbt, corpus, feature_counter):
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split(' '):
            feature_counter[word] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.middle.split(' '):
            feature_counter[word] += 1
    return feature_counter

In [46]:
# define another featurizer -> w2v?

In [127]:
ex = corpus.get_examples_for_entities('Kenya', 'Somalia')
ex

[Example(entity_1='Kenya', entity_2='Somalia', left='The al-Qaida-linked extremist group has vowed retribution on ', mention_1='Kenya', middle=' for sending troops to ', mention_2='Somalia', right=' to fight it.', left_POS='The/DET al/PROPN -/PUNCT Qaida/PROPN -/PUNCT linked/VERB extremist/ADJ group/NOUN has/AUX vowed/VERB retribution/NOUN on/ADP', mention_1_POS='Kenya/PROPN', middle_POS='/SPACE for/ADP sending/VERB troops/NOUN to/PART', mention_2_POS='Somalia/PROPN', right_POS='/SPACE to/PART fight/VERB it/PRON ./PUNCT'),
 Example(entity_1='Kenya', entity_2='Somalia', left='NAIROBI, ', mention_1='Kenya', middle=' A police officer in ', mention_2='Somalia', right=' says a car bomb blast near a security checkpoint at the presidential palace in the capital killed at least two people.', left_POS='NAIROBI/PROPN ,/PUNCT', mention_1_POS='Kenya/PROPN', middle_POS='/SPACE A/DET police/NOUN officer/NOUN in/ADP', mention_2_POS='Somalia/PROPN', right_POS='/SPACE says/VERB a/DET car/NOUN bomb/NOUN

In [128]:
' '.join((ex[0][2], ex[0][3], ex[0][4], ex[0][5], ex[0][6]))

'The al-Qaida-linked extremist group has vowed retribution on  Kenya  for sending troops to  Somalia  to fight it.'

In [129]:
' '.join((ex[1][2], ex[1][3], ex[1][4], ex[1][5], ex[1][6]))

'NAIROBI,  Kenya  A police officer in  Somalia  says a car bomb blast near a security checkpoint at the presidential palace in the capital killed at least two people.'

In [130]:
corpus.show_examples_for_pair('Kenya', 'Somalia')

The first of 2 examples for Kenya and Somalia is:
Example(entity_1='Kenya', entity_2='Somalia', left='The al-Qaida-linked extremist group has vowed retribution on ', mention_1='Kenya', middle=' for sending troops to ', mention_2='Somalia', right=' to fight it.', left_POS='The/DET al/PROPN -/PUNCT Qaida/PROPN -/PUNCT linked/VERB extremist/ADJ group/NOUN has/AUX vowed/VERB retribution/NOUN on/ADP', mention_1_POS='Kenya/PROPN', middle_POS='/SPACE for/ADP sending/VERB troops/NOUN to/PART', mention_2_POS='Somalia/PROPN', right_POS='/SPACE to/PART fight/VERB it/PRON ./PUNCT')


Here's how this featurizer works on a single example:

In [99]:
ij = kb.get_triples_for_entities('Kenya', 'Somalia')
ij

[KBTriple(rel='adjoins', sbj='Kenya', obj='Somalia')]

In [100]:
kbt = ij[0]
kbt

KBTriple(rel='adjoins', sbj='Kenya', obj='Somalia')

In [102]:
corpus.get_examples_for_entities(kbt.sbj, kbt.obj)

[Example(entity_1='Kenya', entity_2='Somalia', left='The al-Qaida-linked extremist group has vowed retribution on ', mention_1='Kenya', middle=' for sending troops to ', mention_2='Somalia', right=' to fight it.', left_POS='The/DET al/PROPN -/PUNCT Qaida/PROPN -/PUNCT linked/VERB extremist/ADJ group/NOUN has/AUX vowed/VERB retribution/NOUN on/ADP', mention_1_POS='Kenya/PROPN', middle_POS='/SPACE for/ADP sending/VERB troops/NOUN to/PART', mention_2_POS='Somalia/PROPN', right_POS='/SPACE to/PART fight/VERB it/PRON ./PUNCT'),
 Example(entity_1='Kenya', entity_2='Somalia', left='NAIROBI, ', mention_1='Kenya', middle=' A police officer in ', mention_2='Somalia', right=' says a car bomb blast near a security checkpoint at the presidential palace in the capital killed at least two people.', left_POS='NAIROBI/PROPN ,/PUNCT', mention_1_POS='Kenya/PROPN', middle_POS='/SPACE A/DET police/NOUN officer/NOUN in/ADP', mention_2_POS='Somalia/PROPN', right_POS='/SPACE says/VERB a/DET car/NOUN bomb/NOUN

In [103]:
simple_bag_of_words_featurizer(kbt, corpus, Counter())

Counter({'': 4,
         'for': 1,
         'sending': 1,
         'troops': 1,
         'to': 1,
         'A': 1,
         'police': 1,
         'officer': 1,
         'in': 1})

You can experiment with adding new kinds of features just by implementing additional featurizers, following `simple_bag_of_words_featurizer` as an example.

Now, in order to apply machine learning algorithms such as those provided by `sklearn`, we need a way to convert datasets of `KBTriple`s into feature matrices. The following steps achieve that: 

In [43]:
# kbts_by_rel, labels_by_rel = dataset.build_dataset()

# featurized = dataset.featurize(kbts_by_rel, featurizers=[simple_bag_of_words_featurizer])

### Experiments

Now we need some functions to train models, make predictions, and evaluate the results. We'll start with `train_models()`. This function takes as arguments a dictionary of data splits, a list of featurizers, the name of the split on which to train (by default, 'train'), and a model factory, which is a function which initializes an `sklearn` classifier (by default, a logistic regression classifier). It returns a dictionary holding the featurizers, the vectorizer that was used to generate the training matrix, and a dictionary holding the trained models, one per relation.

In [131]:
train_result = rel_ext.train_models(
    splits, split_name='train',
    featurizers=[simple_bag_of_words_featurizer])

Next comes `predict()`. This function takes as arguments a dictionary of data splits, the outputs of `train_models()`, and the name of the split for which to make predictions. It returns two parallel dictionaries: one holding the predictions (grouped by relation), the other holding the true labels (again, grouped by relation).

In [132]:
predictions, true_labels = rel_ext.predict(
    splits, train_result, split_name='test')

In [50]:
len(predictions['adjoins'])

3132

In [133]:
splits['train'].count_examples()

                                             examples
relation               examples    triples    /triple
--------               --------    -------    -------
adjoins                     641       1283       0.50
capital                      90        406       0.22
contains                    271      14461       0.02
has_spouse                    4       2419       0.00
nationality                  14       1296       0.01
place_of_birth                0        874       0.00
place_of_death                4        676       0.01
worked_at                     1        898       0.00


In [134]:
splits['test'].count_examples()

                                             examples
relation               examples    triples    /triple
--------               --------    -------    -------
adjoins                     145        419       0.35
capital                      19        116       0.16
contains                     59       4220       0.01
has_spouse                    0        575       0.00
nationality                   1        302       0.00
place_of_birth                1        223       0.00
place_of_death                0        155       0.00
worked_at                     0        252       0.00


Now `evaluate_predictions()`. This function takes as arguments the parallel dictionaries of predictions and true labels produced by `predict()`. It prints summary statistics for each relation, including precision, recall, and F<sub>0.5</sub>-score, and it returns the macro-averaged F<sub>0.5</sub>-score.

In [135]:
rel_ext.evaluate_predictions(predictions, true_labels)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.912      0.912      0.912        419       2795
capital                   0.972      0.905      0.958        116       2492
contains                  0.993      0.990      0.993       4220       6596
has_spouse                0.991      1.000      0.993        575       2951
nationality               0.987      0.997      0.989        302       2678
place_of_birth            0.982      0.996      0.985        223       2599
place_of_death            0.987      1.000      0.990        155       2531
worked_at                 0.984      1.000      0.987        252       2628
------------------    ---------  ---------  ---------  ---------  ---------
macro-average             0.976      0.975      0.976       6262      25270


0.9757934267772528

Finally, we introduce `rel_ext.experiment()`, which basically chains together `rel_ext.train_models()`, `rel_ext.predict()`, and `rel_ext.evaluate_predictions()`. For convenience, this function returns the output of `rel_ext.train_models()` as its result.

Running `rel_ext.experiment()` in its default configuration will give us a baseline result for machine-learned models.

In [136]:
_ = rel_ext.experiment(
    splits,
    featurizers=[simple_bag_of_words_featurizer])

KeyError: 'dev'

Considering how vanilla our model is, these results are quite surprisingly good! We see huge gains for every relation over our `top_k_middles_classifier` from [the previous notebook](rel_ext_01_task.ipynb#A-simple-baseline-model). This strong performance is a powerful testament to the effectiveness of even the simplest forms of machine learning.

But there is still much more we can do. To make further gains, we must not treat the model as a black box. We must open it up and get visibility into what it has learned, and more importantly, where it still falls down.

## Analysis

### Examining the trained models

One important way to gain understanding of our trained model is to inspect the model weights. What features are strong positive indicators for each relation, and what features are strong negative indicators?

In [17]:
rel_ext.examine_model_weights(train_result)

Highest and lowest feature weights for relation adjoins:

     0.778 out
     0.778 carried
     0.778 allies
     ..... .....
    -0.806 a
    -0.885 the
    -1.476 

Highest and lowest feature weights for relation author:

     0.000 out
     0.000 carried
     0.000 allies
     ..... .....
    -0.438 the
    -1.113 ,
    -3.038 

Highest and lowest feature weights for relation capital:

     0.000 out
     0.000 carried
     0.000 allies
     ..... .....
    -0.380 the
    -0.833 ,
    -2.511 

Highest and lowest feature weights for relation contains:

     0.000 thats
     0.000 series
     0.000 presence
     ..... .....
    -0.516 the
    -1.550 ,
    -3.803 

Highest and lowest feature weights for relation film_performance:

     0.000 out
     0.000 carried
     0.000 allies
     ..... .....
    -0.423 the
    -1.034 ,
    -2.894 

Highest and lowest feature weights for relation founders:

     0.000 out
     0.000 carried
     0.000 allies
     ..... .....
    -0.446 the
    -

By and large, the high-weight features for each relation are pretty intuitive — they are words that are used to express the relation in question. (The counter-intuitive results merit a bit of investigation!)

The low-weight features (that is, features with large negative weights) may be a bit harder to understand. In some cases, however, they can be interpreted as features which indicate some _other_ relation which is anti-correlated with the target relation. (As an example, "directed" is a negative indicator for the `author` relation.)

__Optional exercise:__ Investigate one of the counter-intuitive high-weight features. Find the training examples which caused the feature to be included. Given the training data, does it make sense that this feature is a good predictor for the target relation?

<!--
- SPOILER: Using `penalty='l1'` results in somewhat less intuitive feature weights, and about the same performance.
- SPOILER: Using `penalty='l1', C=0.1` results in much more intuitive feature weights, but much worse performance.
-->

### Discovering new relation instances

Another way to gain insight into our trained models is to use them to discover new relation instances that don't currently appear in the KB. In fact, this is the whole point of building a relation extraction system: to extend an existing KB (or build a new one) using knowledge extracted from natural language text at scale. Can the models we've trained do this effectively?

Because the goal is to discover new relation instances which are *true* but *absent from the KB*, we can't evaluate this capability automatically. But we can generate candidate KB triples and manually evaluate them for correctness.

To do this, we'll start from corpus examples containing pairs of entities which do not belong to any relation in the KB (earlier, we described these as "negative examples"). We'll then apply our trained models to each pair of entities, and sort the results by probability assigned by the model, in order to find the most likely new instances for each relation.

In [137]:
rel_ext.find_new_relation_instances(dataset, featurizers=[simple_bag_of_words_featurizer])

Highest probability examples for relation adjoins:

     1.000 KBTriple(rel='adjoins', sbj='Ladkani', obj='about_$3,000')
     1.000 KBTriple(rel='adjoins', sbj='about_$3,000', obj='Ladkani')
     0.999 KBTriple(rel='adjoins', sbj='2011', obj='Sudan')
     0.999 KBTriple(rel='adjoins', sbj='Sudan', obj='2011')
     0.998 KBTriple(rel='adjoins', sbj='China', obj='Taiwanese')
     0.998 KBTriple(rel='adjoins', sbj='Taiwanese', obj='China')
     0.984 KBTriple(rel='adjoins', sbj='2010.Now_Ouyang', obj='China')
     0.984 KBTriple(rel='adjoins', sbj='China', obj='2010.Now_Ouyang')
     0.984 KBTriple(rel='adjoins', sbj='Italy', obj='Catholic')
     0.984 KBTriple(rel='adjoins', sbj='Catholic', obj='Italy')

Highest probability examples for relation capital:

     0.838 KBTriple(rel='capital', sbj='William_Happer', obj='National_Security_Council')
     0.838 KBTriple(rel='capital', sbj='day', obj='Omar_Marrero')
     0.838 KBTriple(rel='capital', sbj='National_Security_Council', obj='Willia

There are actually some good discoveries here! The predictions for the `author` relation seem especially good. Of course, there are also plenty of bad results, and a few that are downright comical. We may hope that as we improve our models and optimize performance in our automatic evaluations, the results we observe in this manual evaluation improve as well.

__Optional exercise:__ Note that every time we predict that a given relation holds between entities `X` and `Y`, we also predict, with equal confidence, that it holds between `Y` and `X`. Why? How could we fix this?

\[ [top](#Relation-extraction-using-distant-supervision) \]